# Import libraries and helpers

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys

In [ ]:
sys.path.append('./helpers_models/')

In [ ]:
sys.path.append('./data_visualization_and_augmentations/')

In [ ]:
sys.path.append('../torch_videovision/')

In [ ]:
sys.path.append('../3D-ResNets-PyTorch/')

In [ ]:
sys.path.append('./important_csvs/')

In [ ]:
from helpers_3d import *

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

# Load Model, change head, freeze body

In [ ]:
options = {
    "model_depth": 50,
    "model": 'resnet',
    "n_classes": 400,
    "n_finetune_classes": 5,
    "resnet_shortcut": 'B',
    "sample_size": (576,704),
    "sample_duration": 16,
    "pretrain_path": '../3D-ResNets-PyTorch/resnet-50-kinetics.pth',
    "no_cuda": False,
    "arch": 'resnet-50',
    "ft_begin_index": 0
}

opts = namedtuple("opts", sorted(options.keys()))

myopts = opts(**options)

model, parameters = generate_model(myopts)

In [ ]:
adaptive_pooling = AdaptiveConcatPool3d()
os.environ['CUDA_VISIBLE_DEVICES']='0,1'
#torch.cuda.empty_cache()
device = torch.device('cuda') 
head = Head()
adaptive_pooling = adaptive_pooling.to(device)
head = head.to(device)
model.module.avgpool = adaptive_pooling
model.module.fc = head

In [ ]:
for param in model.parameters():
    param.requires_grad = False
    
for param in model.module.avgpool.parameters():
    param.requires_grad = True
    
for param in model.module.fc.parameters():
    param.requires_grad = True

In [ ]:
model.module

## Torchsummary

In [ ]:
summary(model.module, (3,16,576,704))

# Load training and validation sets

In [ ]:
tensor_transform = get_tensor_transform('Kinetics')

In [ ]:
train_transform = get_video_transform(2)
valid_transform = get_video_transform(0)

In [ ]:
df = pd.read_csv('./important_csvs/events_with_number_of_frames_stratified.csv')
df = get_df(df, 16, False)

In [ ]:
class_image_paths, end_idx = get_indices(df)

In [ ]:
train_loader = get_loader(16, 2, end_idx, class_image_paths, train_transform, tensor_transform)

In [ ]:
show_batch(train_loader, 2)

In [ ]:
df = pd.read_csv('./important_csvs/events_with_number_of_frames_stratified.csv')
df = get_df(df, 16, True)

In [ ]:
class_image_paths, end_idx = get_indices(df)

In [ ]:
valid_loader = get_loader(16, 2, end_idx, class_image_paths, valid_transform, tensor_transform)

In [ ]:
show_batch(valid_loader, 2)

# Training loop with live losses plot

# Save model

# Load saved Model for more training